In [1]:
import fileio
import pydata

#import numpy as np
#import pandas as pd

In [ ]:
from importlib import reload

In [ ]:
reload(fileio)
reload(shapes)

In [2]:
myfile = fileio.InputData('arecaceae.csv')
myfile.iucnFile('arecaceae_categories.csv')

In [ ]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [ ]:
myar.spp_inclusion(myfile)

In [ ]:
myar.polys

In [ ]:
myar.new_spp_table('test_kba_log.csv')

---

In [8]:
tilas = myfile.getTiles(3, offsetLat=1.5, offsetLon=1.5)

In [ ]:
myfile.index_reg

In [ ]:
for ti in tilas:
    print(myfile.tile2str(ti))

In [9]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 500, 2)

for group in mysols:
    for iso, so in enumerate(group):
        print("Solution {0} of {1}".format(iso, len(group)))
        print(so.score, "-", so.ndmScore, "-", so.aggrScore)
        #print("Islands: ", pydata.islNum(so))
        #if not pydata.isCont(so):
        #    print("NOT CONTINUOUS!!!")
        #print(so.toBitList())
        print(myfile.tile2str(so))
    

Solution 0 of 15
28 - 11.85521381534636 - 51.71042763069272
0 - - - - 
- - 0 0 - 
- 0 0 0 - 
- - 1 0 0 
- 0 0 0 - 
- 0 0 - 0 
- - - - 0 

Solution 1 of 15
20 - 10.50247099250555 - 41.0049419850111
0 - - - - 
- - 0 0 - 
- 0 1 0 - 
- - 0 0 0 
- 0 0 0 - 
- 0 0 - 0 
- - - - 0 

Solution 2 of 15
23 - 10.310854248702526 - 43.62170849740505
0 - - - - 
- - 0 0 - 
- 0 0 0 - 
- - 0 1 0 
- 0 0 0 - 
- 0 0 - 0 
- - - - 0 

Solution 3 of 15
12 - 4.168682839721441 - 20.337365679442883
0 - - - - 
- - 0 0 - 
- 0 0 0 - 
- - 0 0 0 
- 0 1 0 - 
- 0 0 - 0 
- - - - 0 

Solution 4 of 15
15 - 4.041175216436386 - 23.082350432872772
0 - - - - 
- - 0 0 - 
- 0 0 1 - 
- - 0 0 0 
- 0 0 0 - 
- 0 0 - 0 
- - - - 0 

Solution 5 of 15
2 - 2.0 - 6.0
1 - - - - 
- - 0 0 - 
- 0 0 0 - 
- - 0 0 0 
- 0 0 0 - 
- 0 0 - 0 
- - - - 0 

Solution 6 of 15
7 - 1.9072580821812153 - 10.81451616436243
0 - - - - 
- - 0 0 - 
- 0 0 0 - 
- - 0 0 0 
- 0 0 0 - 
- 0 1 - 0 
- - - - 0 

Solution 7 of 15
2 - 1.5 - 5.0
0 - - - - 
- - 0 0 - 
- 0 0 0 

In [ ]:
def overl(l0, l1):
    out = False
    for ic, c in enumerate(l0):
        if c == 1 and l1[ic] == 1:
            out = True
            break
    return out

In [ ]:
visited = [0 for x in mysols]
sets = {}
cc = 0

for isor, soRef in enumerate(mysols):
    if visited[isor] == 0:
        thisgroup = [isor]
        visited[isor] = 1
        for iso, so in enumerate(mysols):
            if iso != isor and visited[iso] == 0:
                addme = True
                for tg in thisgroup:
                    if overl(so.toBitList(), mysols[tg].toBitList()):
                        addme = False
                        break
                if addme:
                    visited[iso] = 1
                    thisgroup.append(iso)
        if len(thisgroup) > 1:
            sets[cc] = thisgroup
            cc += 1

In [ ]:
for iso in sets[0]:
    print(mysols[iso].score, "-", mysols[iso].ndmScore, "-", mysols[iso].aggrScore)
    print(myfile.tile2str(mysols[iso]))

In [ ]:
len(sets[0])

In [ ]:
[x for x in sets.keys() if sets[x] == 0]

In [ ]:
len(sets)

In [ ]:
mysols[0].originY

In [ ]:
mysols[0].originY = -90

In [ ]:
tarbits = [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
           [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]]
counts = {0:0, 1:0, 2:0, 3:0}
#ndmabs = np.linspace(0, 50, 20).tolist()
#ndmouts = np.linspace(0, 1, 20).tolist()
ndmwei = np.linspace(10, 30, 20).tolist()
bffr = "index,weigth,ar0,ar1,ar2,ar3\n"
counter = 0

for w in ndmwei:            
    mysols = pyData.metasearchAlt(tilas, 0.2, 100, 20, w)
    tline = "{0},{1},".format(counter, w)
    for ita , ta in enumerate(tarbits):
        bit = "nan,"
        for iso, so in enumerate(mysols):
            if so.toBitList() == ta:
                counts[ita] += 1
                bit = "{0},".format(iso)
                break
        tline += bit
    tline = tline[:-1]
    bffr += tline + "\n"
    counter += 1
            
#with open('exp0.csv', 'w') as fhandle:
#with open('exp1.csv', 'w') as fhandle:
#    fhandle.write(bffr)

In [ ]:
print(bffr)

In [ ]:
exp0 = pd.read_csv('exp0.csv',index_col=0)

In [ ]:
exp0[['weigth', 'ar0', 'ar1', 'ar2', 'ar3']].groupby('weigth').mean()

In [ ]:
counts

/* keys are indexes in observations vector. Values are vectors integers that 
	associates to KBA criteria. Code options are:  
		0 = A1a  
		1 = A1b  
		2 = A1c  
		3 = A1d  
		4 = A1e  
		5 = B1  
		6 = B2*/ 

In [ ]:
myfile.rows, myfile.cols

In [ ]:
for r in range(myfile.rows):
    for c in range(myfile.cols):
        print("{0:.1f}|".format(myfile.presence_grid[r][c]), end='')
    print('')